In [4]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib
import builtins
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guyyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

SyntaxError: invalid syntax (<ipython-input-6-28a0344c7e14>, line 3)

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [8]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)

In [9]:
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [10]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
# take the 'title' and 'id' or the first 1000 rows and create an RDD from it
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
# take the 'anchor text' and 'id' or the first 1000 rows and create an RDD from it
doc_anchor_text_id_pairs = parquetFile.limit(1000).select("id", "anchor_text").rdd

In [11]:
doc_anchor_text_pairs=doc_anchor_text_id_pairs.map(lambda x:(" ".join([t[1] for t in x[1]]), x[0]))

In [12]:
english_stopwords = frozenset(stopwords.words('english'))
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  my_count=Counter(tokens)
  words_list=[(x,(id,my_count[x])) for x in my_count if x not in english_stopwords]
  return words_list

In [3]:
def doc_NF(text, id):

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  my_count=Counter(tokens)
  # words_list=[(x,(id,my_count[x])) for x in my_count if x not in english_stopwords]
  sum_square=0
  for c in my_count:
    sum_square+=my_count[c]**2
  if sum_square==0:
    return(id,(0,len(text.split(" "))))
  return (id,(1/math.sqrt(sum_square),len(tokens)))

In [14]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  list_sort=sorted(unsorted_pl)
  return list_sort

In [15]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  my_rdd=postings.map(lambda w: (w[0],len(w[1])))
  return my_rdd

In [16]:
import inverted_index_colab
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,base_dir):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  my_rdd=postings.map(lambda x:(token2bucket_id(x[0]),x)).groupByKey().mapValues(list).mapValues(sorted).map(lambda t: inverted_index_colab.InvertedIndex.write_a_posting_list(t,base_dir))
  

  return my_rdd

In [17]:
# create directories for the different indices 
!mkdir body_index title_index anchor_index

In [18]:
# prepare all the data for the body index
##########################################
# calculate df for each term in rhe vocabulary:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>10)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# caculate the NF tns the document length for each document:
nf=doc_text_pairs.map(lambda x: doc_NF(x[0], x[1]))
nf_dict = nf.collectAsMap()
# createing posting list for each term
posting_locs_list = partition_postings_and_write(postings_filtered,'body_index').collect()
# merge the posting locations into a single dict and run more tests
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [19]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_dict
# Add the NF dictionary to the inverted index
inverted_body.NF=nf_dict
# write the global stats out
inverted_body.write_index('body_index', 'body_index')

In [20]:
w2df_dict

{'political': 290,
 'philosophy': 135,
 'movement': 231,
 'sceptical': 12,
 'authority': 172,
 'rejects': 16,
 'involuntary': 12,
 'forms': 286,
 'hierarchy': 37,
 'calls': 105,
 'abolition': 32,
 'state': 468,
 'holds': 126,
 'undesirable': 13,
 'unnecessary': 34,
 'harmful': 25,
 'historically': 136,
 'usually': 364,
 'described': 358,
 'alongside': 121,
 'libertarian': 15,
 'wing': 48,
 'socialism': 20,
 'socialist': 41,
 'strong': 293,
 'historical': 281,
 'association': 231,
 'history': 638,
 'anarchy': 13,
 'goes': 107,
 'back': 412,
 'prehistory': 17,
 'humans': 127,
 'arguably': 38,
 'lived': 204,
 'societies': 55,
 'long': 464,
 'establishment': 102,
 'formal': 141,
 'states': 554,
 'realms': 14,
 'empires': 30,
 'rise': 200,
 'organised': 43,
 'hierarchical': 17,
 'bodies': 115,
 'toward': 162,
 'also': 897,
 'rose': 113,
 '19th': 217,
 'century': 496,
 'emerged': 114,
 'latter': 282,
 'half': 294,
 'first': 805,
 'decades': 162,
 '20th': 226,
 'anarchist': 14,
 'flourished':

In [21]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [22]:
def cos_sim(query_to_search,index):
  ''' A function that compute the cosine similarity of a query to each relevent document
      in the corpus. relevent document is a document with at least 1 common word with the query.
  Parameters:
  -----------
    query_to_search: list of tokens.
    index: inverted index
  
  Returns:
  --------
    list of tuples (id, score) sorted bt score.
  '''
    
  epsilon = .0000001
  docs = len(index.NF)
  sim = defaultdict(list)
  query_terms = Counter(query_to_search)
  for token in np.unique(query_to_search):
      if token in index.df.keys(): #avoid terms that do not appear in the index. 
            q_tf = query_terms[token]/len(query_to_search)
            df=index.df[token]
            idf = math.log((docs)/(df+epsilon),2)
            q_tf_idf=q_tf*idf
            pl=read_posting_list(index,token)
            for d in pl:
              tf=d[1]/(index.NF[d[0]][1])
              tf_idf=tf*idf
              sim[d[0]].append(q_tf_idf*tf_idf)
  for k,v in sim.items():
      valSum=builtins.sum(v)
      norm=index.NF[k][0]/len(query_to_search)
      sim[k]=norm*valSum
  return get_id_title(sorted([(doc_id,score) for doc_id, score in sim.items()], key = lambda x: x[1],reverse=True)[:100])

In [23]:
cos_sim(["alessandro"],inverted_body)


NameError: ignored

In [ ]:
# prepare all the data for the title index
##########################################
# calculate df for each term in rhe vocabulary:
word_counts_t = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_t = word_counts_t.groupByKey().mapValues(reduce_word_counts)
w2df_t = calculate_df(postings_t)
w2df_dict_t = w2df_t.collectAsMap()
# caculate the NF tns the document length for each document:
nf_t=doc_title_pairs.map(lambda x: doc_NF(x[0], x[1]))
nf_dict_t = nf_t.collectAsMap()
# save doc id and title for each documet
title_id_tup=doc_title_pairs.map(lambda x:(x[1],x[0])).collectAsMap()
# createing posting list for each term
posting_locs_list_t = partition_postings_and_write(postings_t,'title_index').collect()
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs_t = defaultdict(list)
for posting_loc in posting_locs_list_t:
  for k, v in posting_loc.items():
    super_posting_locs_t[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_t
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_t
# Add the NF dictionary to the inverted index
inverted_title.NF=nf_dict_t
# Add the title_dict to the inverted index
inverted_title.title_dict=title_id_tup
# write the global stats out
inverted_title.write_index('title_index', 'title_index')

In [ ]:
w2df_dict_t

In [ ]:
def title_binary_rank(query_to_search,index):

    rank = Counter()
    for token in np.unique(query_to_search):
        if token in index.df.keys(): #avoid terms that do not appear in the index. 
            pl=read_posting_list(index,token)
            for d in pl:
              rank[d[0]]+=1
    return get_id_title([(l,k) for k,l in sorted([(j,i) for i,j in rank.items()], reverse=True)])

In [ ]:
title_binary_rank(["alessandro"],inverted_title)

In [ ]:
def get_id_title(list):
  return [(id,inverted_title.title_dict[id]) for id,x in list]

In [ ]:
# prepare all the data for the anchor index
##########################################
# calculate df for each term in rhe vocabulary:
word_counts_a = doc_anchor_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_a = word_counts_a.groupByKey().mapValues(reduce_word_counts)
w2df_a = calculate_df(postings_a)
w2df_dict_a = w2df_t.collectAsMap()
# caculate the NF tns the document length for each document:
nf_a=doc_anchor_text_pairs.map(lambda x: doc_NF(x[0], x[1]))
nf_dict_a = nf_a.collectAsMap()
# createing posting list for each term
posting_locs_list_a = partition_postings_and_write(postings_a,'anchor_index').collect()
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs_a = defaultdict(list)
for posting_loc in posting_locs_list_a:
  for k, v in posting_loc.items():
    super_posting_locs_a[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_a
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict_a
# Add the NF dictionary to the inverted index
inverted_anchor.NF=nf_dict_a
# write the global stats out
inverted_anchor.write_index('anchor_index', 'anchor_index')

In [ ]:
title_binary_rank(["alessandro"],inverted_anchor)

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  ''' 
  vertices_id=pages.map(lambda x: (x[0],))
  rdd_anchor=pages.flatMap(lambda x: (x[1]))
  rdd_id=rdd_anchor.map(lambda x: (x[0],))
  all_vertices=vertices_id.union(rdd_id)
  vertices=all_vertices.distinct()

  v=pages.flatMapValues(lambda x: x)
  edges=v.map(lambda x: (x[0],x[1][0])).distinct()

  return edges, vertices

In [ ]:
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(doc_anchor_text_id_pairs)
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())

In [ ]:
pagerank_df=pr.toPandas()
pagerank_dict=pagerank_df.set_index('id')['pagerank'].to_dict()
# page rank for each document
inverted_anchor.page_score=pagerank_dict

In [ ]:
def get_pagerank(id_list):
  return [inverted_anchor.page_score[i] for i in id_list]

In [ ]:
list_id=[x for x,y in cos_sim(["alessandro"],inverted_body)]
get_pagerank(list_id)

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

In [ ]:
with open(pv_clean, 'rb') as f:
  wid2pv = pickle.loads(f.read())

In [ ]:
# page views from Augoust 21, saved in inverted_anchor
inverted_anchor.page_views=wid2pv

In [ ]:
def get_views(id_list):
  return [inverted_anchor.page_views[i] for i in id_list]